Date Created - 6th November 2020

Date Updates - 9th November 2020

Created by - Akshay Chougule

Change log
- Create cohort for comparison

## Basics

In [1]:
import pandas as pd
import numpy as np
import requests
import datetime
import json
# from pandas.io.json import json_normalize
# import xlrd

In [2]:
import plotly.express as px 
import plotly.graph_objects as go

import dash  
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import dash_table_experiments as dt
import dash_bootstrap_components as dbc
import dash_table
import dash_daq as daq

In [3]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

## AE Fetching Functions

In [4]:
def get_data(query_url):
    r = requests.get(query_url)
    j = json.loads(r.content)
    return(pd.DataFrame(j['results']))

def get_ae_pt_data(drug_brand_name):
    male_serious_url = f'https://api.fda.gov/drug/event.json?search=patient.drug.openfda.brand_name:%22{drug_brand_name}%22+AND+serious:1+AND+patient.patientsex:1+AND+receivedate:[2013-01-01+TO+2018-12-31]&count=patient.reaction.reactionmeddrapt.exact&limit=1000'
    female_serious_url = f'https://api.fda.gov/drug/event.json?search=patient.drug.openfda.brand_name:%22{drug_brand_name}%22+AND+serious:1+AND+patient.patientsex:0+AND+receivedate:[2013-01-01+TO+2018-12-31]&count=patient.reaction.reactionmeddrapt.exact&limit=1000'
    male_nonserious_url = f'https://api.fda.gov/drug/event.json?search=patient.drug.openfda.brand_name:%22{drug_brand_name}%22+AND+serious:2+AND+patient.patientsex:1+AND+receivedate:[2013-01-01+TO+2018-12-31]&count=patient.reaction.reactionmeddrapt.exact&limit=1000'
    female_nonserious_url = f'https://api.fda.gov/drug/event.json?search=patient.drug.openfda.brand_name:%22{drug_brand_name}%22+AND+serious:2+AND+patient.patientsex:0+AND+receivedate:[2013-01-01+TO+2018-12-31]&count=patient.reaction.reactionmeddrapt.exact&limit=1000'
    df_male_ser = get_data(male_serious_url)
    df_male_ser['gender']='male'
    df_male_ser['serious']=1
    df_female_ser = get_data(female_serious_url)
    df_female_ser['gender']='female'
    df_female_ser['serious']=1
    df_male_nonser = get_data(male_nonserious_url)
    df_male_nonser['gender']='male'
    df_male_nonser['serious']=0
    df_female_nonser = get_data(female_nonserious_url)
    df_female_nonser['gender']='female'
    df_female_nonser['serious']=0
    return(pd.concat([df_male_ser, df_female_ser,df_male_nonser,df_female_nonser], ignore_index=True))


def get_faers_data(drug_brand_name,sex,serious,meddra):
    query_url = f'https://api.fda.gov/drug/event.json?search=patient.drug.openfda.brand_name:%22{drug_brand_name}%22+AND+serious:{serious}+AND+patient.patientsex:{sex}+AND+receivedate:[2013-01-01+TO+2018-12-31]&count=patient.reaction.reactionmeddra{meddra}.exact&limit=1000'
    print(query_url)
    res_df = get_data(query_url)
    return(res_df)

def get_ae_soc_data(drug_brand_name):
    df_soc = pd.read_csv('dummy_safety_soc.csv')
    soc_copy = df_soc.copy()
    max_count = soc_copy.total_from_api.max()
    soc_copy['percent_total_api'] = round(100*(soc_copy.total_from_api/max_count),0)
    soc_copy['percent_serious_api'] = round(100*(soc_copy.serious_from_api/max_count),0)
    max_count = soc_copy.related_from_ct.max()
    soc_copy['percent_related_ct'] = round(100*(soc_copy.related_from_ct/max_count),0)
    soc_copy['percent_serious_ct'] = round(100*(soc_copy.serious_from_ct/max_count),0)
    soc_copy.sort_values('percent_total_api', ascending=False, inplace=True)
    return(soc_copy)

## Testing

In [5]:
get_faers_data("avastin",1,2,'pt')

https://api.fda.gov/drug/event.json?search=patient.drug.openfda.brand_name:%22avastin%22+AND+serious:2+AND+patient.patientsex:1+AND+receivedate:[2013-01-01+TO+2018-12-31]&count=patient.reaction.reactionmeddrapt.exact&limit=1000


,term,count
0,DRUG INEFFECTIVE,112
1,OFF LABEL USE,90
2,DISEASE PROGRESSION,70
3,FATIGUE,69
4,NO ADVERSE EVENT,51
...,...,...
600,VENOUS THROMBOSIS,1
601,VISUAL FIELD TESTS ABNORMAL,1
602,WHEEZING,1
603,WHITE BLOOD CELL COUNT INCREASED,1


## Dash App

In [6]:
tabs_styles = {
    'height': '51px'
}
tab_style = {
    'borderBottom': '1px solid #d6d6d6',
    'padding': '2px',
    'fontWeight': 'bold'
}

tab_selected_style = {
    'borderTop': '1px solid #d6d6d6',
    'borderBottom': '5px solid orange',
    'backgroundColor': '#4c2d4dff',
    'color': 'white',
    'padding': '10px'
}

In [18]:
df = get_faers_data('avastin',1,1,'pt')
fig = px.bar(df, x='term', y='count')
faers_tab = fig.show()

https://api.fda.gov/drug/event.json?search=patient.drug.openfda.brand_name:%22avastin%22+AND+serious:1+AND+patient.patientsex:1+AND+receivedate:[2013-01-01+TO+2018-12-31]&count=patient.reaction.reactionmeddrapt.exact&limit=1000


In [38]:
race = dbc.FormGroup([
                dbc.Label("Race", html_for="dropdown"),
                dcc.Dropdown(id="select_race",
                 options=[
                     {"label": "White", "value": "white"},
                     {"label": "Black", "value": "black"},
                     {"label": "Asian", "value": "asian"},
                     {"label": "Pacific Islander", "value": "islander"},
                     {"label": "Other or Unknown", "value": "other"},
                     ],
                     multi=True
                             ),
])

sex = dbc.FormGroup([
                dbc.Label("Sex", html_for="dropdown"),
                dcc.Dropdown(id="select_sex",
                 options=[
                     {"label": "Any", "value": "all"},
                     {"label": "Male", "value": "male"},
                     {"label": "Female", "value": "female"},
                     ],
                    value='male'),
])


drug = dbc.FormGroup([
                dbc.Label("Drug Name", html_for="dropdown"),
                dcc.Dropdown(id="select_drug",
                 options=[
                     {"label": "Avastin", "value": "avastin"},
                     {"label": "Docetaxel", "value": "docetaxel"},
                     {"label": "Carboplatin", "value": "carboplatin"},
                     ],
                    value='avastin'),
])


drug_class = dbc.FormGroup([
                dbc.Label("Drug Name or Class", html_for="dropdown"),
                dcc.Dropdown(id="select_drugclass",
                 options=[
                     {"label": "Avastin", "value": "avastin"},
                     {"label": "Docetaxel", "value": "docetaxel"},
                     {"label": "Carboplatin", "value": "carboplatin"},
                     ],
                    value='avastin'),
])

data_sources = dbc.FormGroup([
                dbc.Label("Data Sources", html_for="dropdown"),
                dcc.Dropdown(id="select_source",
                 options=[
                     {"label": "FAERS", "value": "faers"},
                     {"label": "MEDS (Clinical Trials)", "value": "trials"},
                     {"label": "Quantum (RWD)", "value": "rwd"},
                     {"label": "EudraVigilance", "value": "eudravigilance"},
                     {"label": "VigiBase", "value": "vigibase"},
                     ],
                    value='avastin'),
])

serious = dbc.FormGroup([
                dbc.Label("Serious", html_for="dropdown"),
                dcc.Dropdown(id="select_serious",
                 options=[
                     {"label": "Serious", "value": "serious"},
                     {"label": "Non-serious", "value": "nonserious"},
                     ],
                     value='serious'
                     #,multi=True
                             ),
])


meddra = dbc.FormGroup([
                dbc.Label("MedDRA Level", html_for="dropdown"),
                dcc.Dropdown(id="select_meddra",
                 options=[
                     {"label": "System Organ Class (SOC)", "value": "soc"},
                     {"label": "Preferred Term (PT)", "value": "pt"},
                     ],
                     value='soc'
                     #,multi=True
                             ),
])

data_switch = dbc.FormGroup([
                dbc.Label("Data Table to Viz"),
                daq.ToggleSwitch(
                    id='view_data',
                    color='#4c2d4dff',
                    value=False
                )
])

meddra_switch = dbc.FormGroup([
                dbc.Label("SOC to PT"),
                daq.ToggleSwitch(
                    id='view_pt',
                    color='#4c2d4dff',
                    value=False
                )
])


date_slider = dcc.DatePickerRange(
    start_date_placeholder_text="Start Period",
    end_date_placeholder_text="End Period",
    calendar_orientation='vertical',
)

In [39]:
drug_class1 = dbc.FormGroup([
                dbc.Label("Drug Name or Class", html_for="dropdown"),
                dcc.Dropdown(id="select_drugclass_c1",
                 options=[
                     {"label": "Avastin", "value": "avastin"},
                     {"label": "Docetaxel", "value": "docetaxel"},
                     {"label": "Carboplatin", "value": "carboplatin"},
                     ],
                    value='avastin'),
])

data_sources1 = dbc.FormGroup([
                dbc.Label("Data Sources", html_for="dropdown"),
                dcc.Dropdown(id="select_source_c1",
                 options=[
                     {"label": "FAERS", "value": "faers"},
                     {"label": "MEDS (Clinical Trials)", "value": "trials"},
                     {"label": "Quantum (RWD)", "value": "rwd"},
                     {"label": "EudraVigilance", "value": "eudravigilance"},
                     {"label": "VigiBase", "value": "vigibase"},
                     ],
                    value='avastin'),
])

serious1 = dbc.FormGroup([
                dbc.Label("Serious", html_for="dropdown"),
                dcc.Dropdown(id="select_serious_c1",
                 options=[
                     {"label": "Serious", "value": "serious"},
                     {"label": "Non-serious", "value": "nonserious"},
                     ],
                     value='serious'
                     #,multi=True
                             ),
])


meddra1 = dbc.FormGroup([
                dbc.Label("MedDRA Level", html_for="dropdown"),
                dcc.Dropdown(id="select_meddra_c1",
                 options=[
                     {"label": "System Organ Class (SOC)", "value": "soc"},
                     {"label": "Preferred Term (PT)", "value": "pt"},
                     ],
                     value='soc'
                     #,multi=True
                             )
]),

sex1 = dbc.FormGroup([
                dbc.Label("Sex", html_for="dropdown"),
                dcc.Dropdown(id="select_sex_c1",
                 options=[
                     {"label": "Any", "value": "all"},
                     {"label": "Male", "value": "male"},
                     {"label": "Female", "value": "female"},
                     ],
                    value='male'),
]),
    
drug_class2 = dbc.FormGroup([
                dbc.Label("Drug Name or Class", html_for="dropdown"),
                dcc.Dropdown(id="select_drugclass_c2",
                 options=[
                     {"label": "Avastin", "value": "avastin"},
                     {"label": "Docetaxel", "value": "docetaxel"},
                     {"label": "Carboplatin", "value": "carboplatin"},
                     ],
                    value='avastin'),
]),

data_sources2 = dbc.FormGroup([
                dbc.Label("Data Sources", html_for="dropdown"),
                dcc.Dropdown(id="select_source_c2",
                 options=[
                     {"label": "FAERS", "value": "faers"},
                     {"label": "MEDS (Clinical Trials)", "value": "trials"},
                     {"label": "Quantum (RWD)", "value": "rwd"},
                     {"label": "EudraVigilance", "value": "eudravigilance"},
                     {"label": "VigiBase", "value": "vigibase"},
                     ],
                    value='avastin'),
])

serious2 = dbc.FormGroup([
                dbc.Label("Serious", html_for="dropdown"),
                dcc.Dropdown(id="select_serious_c2",
                 options=[
                     {"label": "Serious", "value": "serious"},
                     {"label": "Non-serious", "value": "nonserious"},
                     ],
                     value='serious'
                     #,multi=True
                             ),
])


meddra2 = dbc.FormGroup([
                dbc.Label("MedDRA Level", html_for="dropdown"),
                dcc.Dropdown(id="select_meddra_c2",
                 options=[
                     {"label": "System Organ Class (SOC)", "value": "soc"},
                     {"label": "Preferred Term (PT)", "value": "pt"},
                     ],
                     value='soc'
                     #,multi=True
                             ),
])

sex2 = dbc.FormGroup([
                dbc.Label("Sex", html_for="dropdown"),
                dcc.Dropdown(id="select_sex_c2",
                 options=[
                     {"label": "Any", "value": "all"},
                     {"label": "Male", "value": "male"},
                     {"label": "Female", "value": "female"},
                     ],
                    value='male'),
]),

In [ ]:
app = dash.Dash(__name__)

# App layout
app.layout = html.Div([
    
    html.H1("Safety Data Explorer", style={'text-align': 'center',
                                           'padding': '4px',
                                           #'backgroundColor': '#4c2d4dff',
                                           'color': '#4c2d4dff'}),
    
    html.Br(),   
    
    # Div for all tabs   
    html.Div(dcc.Tabs([
        
        ### Tab 1 ###
        dcc.Tab(label='Explore Safety Data', children=[
            
            ### Tab 1 - col 1 ###
            html.Div([html.Br(),
                    drug,
                    html.Br(),
                    data_sources,
                    html.Br(),
                    html.H4("Select filters", style={'text-align': 'center','color': '#4c2d4dff'}),
                    #html.Br(),
                    serious, html.Br(),
                    sex, html.Br(),
                    race, html.Br(),
                    dbc.Label("Date Range", html_for="range-slider"),
                    date_slider, html.Br(), html.Br(),
                    html.H4("Change View", style={'text-align': 'center','color': '#4c2d4dff'}), html.Br(),
                    data_switch, html.Br(),
                    meddra_switch
            ], style={'width': '13%', 'display': 'inline-block'}),
            
            ### Tab 1 - col 2 ###
            html.Div([], style={'width': '3%', 'display': 'inline-block'}),
            
            ### Tab 1 - col 3 ###
            html.Div(id='output-sae-subs', style={'width': '84%', 'display': 'inline-block'}),
            
        ], style=tab_style, selected_style=tab_selected_style),
        
        
        ### Tab 2 ###
        dcc.Tab(label='Compare Safety Data', children=[
            ### Tab 2 - col 1 ###
            html.Div([html.Br(),
                    drug_class1,
                    html.Br(),
                    data_sources1,
                    html.Br(),
                    html.H4("Select filters", style={'text-align': 'center','color': '#4c2d4dff'}),
                    #html.Br(),
                    serious1, html.Br(),
                    sex1, html.Br(),
                    #race, html.Br(),
                    #dbc.Label("Date Range", html_for="range-slider"),
                    #date_slider, html.Br(), html.Br(),
                    #html.H4("Change View", style={'text-align': 'center','color': '#4c2d4dff'}), html.Br(),
                    #data_switch, html.Br(),
                    #meddra_switch
            ], style={'width': '10%', 'display': 'inline-block'}),
            
            ### Tab 2 - col 2 ###
            html.Div([], style={'width': '3%', 'display': 'inline-block'}),
            
            ### Tab 2 - col 3 ###
#             html.Div([html.Br(),
#                     drug,
#                     html.Br(),
#                     data_sources,
#                     html.Br(),
#                     html.H4("Select filters", style={'text-align': 'center','color': '#4c2d4dff'}),
#                     #html.Br(),
#                     serious, html.Br(),
#                     sex, html.Br(),
#                     race, html.Br(),
#                     dbc.Label("Date Range", html_for="range-slider"),
#                     date_slider, html.Br(), html.Br(),
#                     html.H4("Change View", style={'text-align': 'center','color': '#4c2d4dff'}), html.Br(),
#                     data_switch, html.Br(),
#                     meddra_switch
#             ], style={'width': '10%', 'display': 'inline-block'}),
            
            ### Tab 2 - col 4 ###
            html.Div([], style={'width': '77%', 'display': 'inline-block'}),
            
     ], style=tab_style, selected_style=tab_selected_style),

    ])) # Div for all tabs end here
    
])


# ------------------------------------------------------------------------------
# Connect the data with Dash Components
@app.callback(
    [dash.dependencies.Output('output-sae-subs', 'children'),
     dash.dependencies.Output('output-oae-subs', 'children'),
     dash.dependencies.Output('soc-tab', 'children'),
     dash.dependencies.Output('soc-viz', 'children')],
    [Input(component_id='select_drug', component_property='value'),
     Input(component_id='select_source', component_property='value'),
     Input(component_id='select_sex', component_property='value'),
     Input(component_id='select_serious', component_property='value'),
     Input(component_id='view_data', component_property='value'),
     Input(component_id='view_pt', component_property='value')]
)


# update logic
def update_graph(drug_name, select_sex, select_serious, view_data, view_pt):
    print(drug_name, select_sex, select_serious, view_data, view_pt)  
    #male ['serious'] True True
    
    if select_sex=='male':
        sex=1
    if select_sex=='female':
        sex=0

    if view_pt=='True':
        meddra='soc'
    else:
        meddra='pt'
    
    if select_serious=='serious':
        serious='1'
    if select_serious=='nonserious':
        serious='2'
    
    df = get_faers_data(drug_name,sex,serious,meddra)
    
    print(df.head())
    
    # The data table view
    if view_data == False:
        print('reached inner loop')

        faers_data = dash_table.DataTable(
                        id='table',
                        columns=[{"name": i, "id": i} for i in df.columns],
                        data=df.to_dict("rows"),
                        filter_action="native",
                        export_format="csv",
                        export_headers="display",
                        style_cell={'width': '300px',
                        'height': '60px',
                        'textAlign': 'left'},
                        style_data_conditional=[{
                            'if': {'row_index': 'odd'},
                            'backgroundColor': 'rgb(248, 248, 248)'
                        }],
                        style_header={
                            'backgroundColor': 'rgb(230, 230, 230)',
                            'fontWeight': 'bold'
                        })
        faers_tab = html.Div([
            html.H4(' ', style={'text-align': 'center'}),
            faers_data
        ])
       
        ct_tab=dash_table.DataTable(
                    id='table2',
                    columns=[{"name": i, "id": i} for i in df.columns],
                    data=df.to_dict("rows"),
                    filter_action="native",
                    export_format="csv",
                    export_headers="display",
                    style_cell={'width': '300px',
                    'height': '60px',
                    'textAlign': 'left'},
                    style_data_conditional=[{
                        'if': {'row_index': 'odd'},
                        'backgroundColor': 'rgb(248, 248, 248)'
                    }],
                    style_header={
                        'backgroundColor': 'rgb(230, 230, 230)',
                        'fontWeight': 'bold'
                    })
    
    if view_data == True:
        print('reached the else part ##########')
        fig = px.bar(df, x='term', y='count')
        faers_tab = html.Div([
            dcc.Graph(figure=fig)
        ])
        fig2 = px.bar(df, x='term', y='count')
        ct_tab = html.Div([
            dcc.Graph(figure=fig2)
        ])
    
    
    soc_data = get_ae_soc_data('test')
    df = soc_data.head(10)
    categories = df.Total

    fig = go.Figure()

    fig.add_trace(go.Scatterpolar(
          #r=[1, 5, 2, 2, 3],
          r=df.percent_total_api,
          theta=categories,
          fill='toself',
          name='FAERS'
    ))
    fig.add_trace(go.Scatterpolar(
          r=df.percent_related_ct,
          theta=categories,
          fill='toself',
          name='Clinical Trials'
    ))

    fig.update_layout(
      polar=dict(
        radialaxis=dict(
          visible=True,
          range=[0, 100]
        )),
      showlegend=True
    )
    
    soc_viz = html.Div([
        dcc.Graph(figure=fig)
    ])
    
    #soc_data = soc_data[['Total','related_from_ct', 'serious_from_ct', 'total_from_api', 'serious_from_api', 'percent_total_api', 'percent_serious_api', 'percent_related_ct', 'percent_serious_ct']]
    
    soc_data.rename(columns={'Total':' SOC Term',
                     'related_from_ct':' Related (CT)', 
                     'serious_from_ct':' Related & Serious (CT)', 
                     'total_from_api':' Total (FAERS)', 
                     'serious_from_api':' Serious (FAERS)',
                     'percent_total_api':' Total (FAERS)', 
                     'percent_serious_api':' Total (FAERS)',
                     'percent_related_ct':' Related - Scaled (CT)',
                     'percent_serious_ct':' Related & Serious - Scaled(CT)'},
                    inplace=True)
    
    soc_tab = dash_table.DataTable(
                id='table2',
                columns=[{"name": i, "id": i} for i in soc_data.columns],
                data=soc_data.to_dict("rows"),
                filter_action="native",
                style_cell={'width': '300px',
                'height': '60px',
                'textAlign': 'left'},
                style_data_conditional=[{
                    'if': {'row_index': 'odd'},
                    'backgroundColor': 'rgb(248, 248, 248)'
                }],
                style_header={
                    'backgroundColor': 'rgb(230, 230, 230)',
                    'fontWeight': 'bold'
                })

    return faers_tab, html.Div([
        html.H4(' ', style={'text-align': 'center'}),
        ct_tab
    ]), html.Div([
        html.H4(' ', style={'text-align': 'center'}),
        soc_tab
    ]), soc_viz
    


# ------------------------------------------------------------------------------
if __name__ == '__main__':
    app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [14/Nov/2020 22:53:51] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [14/Nov/2020 22:53:51] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [14/Nov/2020 22:53:51] "GET /_dash-layout HTTP/1.1" 200 -
